# Lecture 13 - PCR and PLS
## CMSE 381 - SS 2024




In [ ]:
# Everyone's favorite standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import time

import seaborn as sns

# ML imports we've used previously
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error



# 1. PCA on Penguins
![Palmer Penguins Picture](https://allisonhorst.github.io/palmerpenguins/reference/figures/lter_penguins.png)

*Artwork by @allison_horst*


For this lab, we are going to again use the <a href = "https://allisonhorst.github.io/palmerpenguins/">Palmer Penguins</a> data set by Allison Horst, Alison Hill, and Kristen Gorman. You should have done this in a previous notebook, but if you don't have the package installed to get the data, you can run 
```
pip install palmerpenguins
```
to have access to the data. 

In [ ]:
from palmerpenguins import load_penguins
penguins = load_penguins()
penguins = penguins.dropna()

#Shuffle the data
#penguins = penguins.sample(frac=1)
penguins.head()


Before we get to the full version, let's just take a look at two of the columns: flipper length and bill length. A nice thing we can do is to also color the data by the species label. 

In [ ]:
sns.scatterplot(x = penguins.bill_length_mm, 
                y = penguins.flipper_length_mm, 
                hue = penguins.species)

Before we get to it, we're going to just work with the columns that are numeric.  

In [ ]:
penguins_num = penguins.select_dtypes(np.number)
penguins_num.head()

We normalize the data to make the visualization easier (meaning shifting our data to have mean 0 in every column, and have standard deviation 1). 

In [ ]:
p_normalized = (penguins_num - penguins_num.mean())/penguins_num.std()
p_normalized.head()

## PCA with just two input columns

To try to draw pictures similar to what we just saw on the slides, we'll first focus on two of the columns. 

In [ ]:
penguins_subset2 = p_normalized[['bill_length_mm', 'flipper_length_mm']]
penguins_subset2

We run PCA using the `PCA` command from `scikitlearn`.

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
# Set up the PCA object
pca = PCA(n_components=2)

# Fit it using our data
pca.fit(penguins_subset2.values)

In [ ]:
pca_df = pca.fit_transform(penguins_subset2.values)
plt.scatter(pca_df[:,0], pca_df[:,1])

The `pca.components_` store information about the lines we are going to project our data onto. Specifically, each row gives us one of these lines.

In [ ]:
pca.components_

In [ ]:
sns.scatterplot(data = penguins_subset2, 
                x = 'bill_length_mm', 
                y = 'flipper_length_mm', 
                hue = penguins.species)

for i, comp in enumerate(pca.components_):
    slope = comp[1]/comp[0]
    plt.plot(np.array([-2,2]), slope*np.array([-2,2]))
    
plt.axis('square')

A common way to look at the relative importance of the PC's is to draw these components as vectors with length based on the explained variance. 

In [ ]:
pca.explained_variance_

In [ ]:
sns.scatterplot(data = penguins_subset2, 
                x = 'bill_length_mm', 
                y = 'flipper_length_mm', 
                hue = penguins.species)

for i, (comp, var) in enumerate(zip(pca.components_, pca.explained_variance_)):
    slope = comp[1]/comp[0]
    plt.plot(np.array([-2,2]), slope*np.array([-2,2]))
    
    comp = comp * var  # scale component by its variance explanation power
    plt.plot(
        [0, comp[0]],
        [0, comp[1]],
        label=f"Component {i}",
        linewidth=5,
        color=f"C{i + 2}",
    )

plt.axis('square')

The next important part are the PC's, which we can get from the `pca` object as follows. I'm going to put them in a dataframe to make drawing and visualization easier. Basically, $PC_1$ is our $Z_1$ in the slides, and $PC_2$ is the $Z_2$.

In [ ]:
# The transform function takes in bill,flipper data points, 
# and returns a PC1,PC2 coordinate for each one. 
penguins_pca = pca.fit_transform(penguins_subset2)
penguins_pca = pd.DataFrame(data = penguins_pca, columns = ['PC1', 'PC2'])
penguins_pca.shape

This is the scatterplot of the data points transformed into the PC space. 

In [ ]:
sns.scatterplot(data = penguins_pca, x = 'PC1', y = 'PC2',hue = penguins.species)

&#9989; **<font color=red>Do this:</font>** What are the PC scores for the first data point (index 0)?  


In [ ]:
# Your answer here


## Penguins PCA with all columns

We used only two columns above for visualization, but we can instead use all the input columns to run our PCA. 

In [ ]:
penguins_num.head()

In [ ]:
pca = PCA(n_components=4)
penguins_pca_all = pca.fit_transform(penguins_num)
penguins_pca_all = pd.DataFrame(data = penguins_pca_all, 
                                columns = ['PC1', 'PC2', 'PC3', 'PC4'])
penguins_pca_all

&#9989; **<font color=red>Do this:</font>** Make a scatter plot of PC1 and PC2 using this new model, and again color the points by `penguins.species`. What do you notice about how the PC plot has changed from the previous setting? 

In [ ]:
# your code here

# Principal Component Regression 

Ok, let's take a hard left turn and go try out some of the dimension reduction methods from Section 6.3. `Scikit-learn` doesn't have a built in function to do PCR (aka PCA and then regression) but it's just as easy for us to do it ourselves. 

PCR is a supervised learning problem, so unlike PCA, it requires the exitence of the response variable. 

Let us shift gear and try to predict ``flipper_length`` from the rest of the numerical variables 



In [ ]:
y = penguins['flipper_length_mm']
X = penguins_num.drop(columns=['flipper_length_mm'])

For the new dataframe `X`, let us calculate its PCs. 

In [ ]:
print(X.shape)
pca = PCA()
X_pca = pca.fit_transform(X)
print(X_pca.shape)

From the output of the above cell, we see that just doing PCA does not automatically reduce the number of predictors. We actually have to manually reduce the predictors by picking how many PCs we want to use in the regression. Now, say we want to reduce the number of predictors from 4 to 3, then I would use only the first 3 PCs for regression.

In [ ]:
from sklearn.linear_model import LinearRegression

regr = LinearRegression()
regr.fit(X_pca[:,:3], y)
mean_squared_error(y,regr.predict(X_pca[:,:3]))

&#9989; **<font color=red>Do this:</font>** My code above contains the rookie mistake of only reporting training error. Write modified code to return the 10-fold CV error of linear regression on the first 3 PCs.

In [ ]:
# Your code here #



# You'll probably want this....
from sklearn.model_selection import cross_val_score


